In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')

##BEST MODEL UP TO TODAY

In [2]:
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Utils.utils import create_URM, create_ICM, combine_matrices
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

##CREATE URM,ICM, ICM STACKED COMPLETELY TO TRAIN THE SINGLE RECOMMENDERS, AND STACKED PARTIALLY TO TUNE THE HYBRID
URM = create_URM()
ICM = create_ICM()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
combined_matrices = combine_matrices(ICM=ICM, URM=URM_train)
ICM_combined = combine_matrices(ICM=ICM,URM=URM)

### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=ICM_combined)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=ICM_combined)
IALS_recommender.fit(epochs=50, num_factors=40, alpha=0.7917156202136513, reg=1.0)

##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=ICM_combined)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

hyperparameters_range_dictionary = {
    "alpha": Real(0.0, 1.0),
    "beta": Real(0.0, 1.0),
    "gamma": Real(0.0, 1.0)
}
recommenders = [RP3Beta_recommender, IALS_recommender, SLIME_recommender]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[combined_matrices, recommenders],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeThreeModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 70
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeThreeModelsByScores.RECOMMENDER_NAME,
                    save_model="best"
                    )



EvaluatorHoldout: Ignoring 13640 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13628 ( 0.2%) Users that have less than 1 test interactions
IALSRecommenderLinear: Epoch 1 of 50. Elapsed time 4.81 sec
IALSRecommenderLinear: Epoch 2 of 50. Elapsed time 9.72 sec
IALSRecommenderLinear: Epoch 3 of 50. Elapsed time 14.52 sec
IALSRecommenderLinear: Epoch 4 of 50. Elapsed time 19.41 sec
IALSRecommenderLinear: Epoch 5 of 50. Elapsed time 24.23 sec
IALSRecommenderLinear: Epoch 6 of 50. Elapsed time 29.13 sec
IALSRecommenderLinear: Epoch 7 of 50. Elapsed time 34.08 sec
IALSRecommenderLinear: Epoch 8 of 50. Elapsed time 38.98 sec
IALSRecommenderLinear: Epoch 9 of 50. Elapsed time 44.02 sec
IALSRecommenderLinear: Epoch 10 of 50. Elapsed time 49.07 sec
IALSRecommenderLinear: Epoch 11 of 50. Elapsed time 54.23 sec
IALSRecommenderLinear: Epoch 12 of 50. Elapsed time 59.52 sec
IALSRecommenderLinear: Epoch 13 of 50. Elapsed time 1.08 min
IALSRecommenderLinear: Epoch 14 

100%|████████████████████████████████████▉| 18032/18059 [11:17<00:00, 47.69it/s]

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.2560066241646047, 'beta': 0.6883207296131294, 'gamma': 0.12426050049505158}


100%|████████████████████████████████████▉| 18056/18059 [11:30<00:00, 47.69it/s]

EvaluatorHoldout: Processed 13628 (100.0%) in 15.90 sec. Users per second: 857
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.2560066241646047, 'beta': 0.6883207296131294, 'gamma': 0.12426050049505158} - results: PRECISION: 0.2366818, PRECISION_RECALL_MIN_DEN: 0.2410143, RECALL: 0.0678853, MAP: 0.1151143, MAP_MIN_DEN: 0.1169211, MRR: 0.4615044, NDCG: 0.2451573, F1: 0.1055085, HIT_RATE: 0.9125330, ARHR_ALL_HITS: 0.7266670, NOVELTY: 0.0055543, AVERAGE_POPULARITY: 0.5331289, DIVERSITY_MEAN_INTER_LIST: 0.9111257, DIVERSITY_HERFINDAHL: 0.9911059, COVERAGE_ITEM: 0.0603023, COVERAGE_ITEM_CORRECT: 0.0467911, COVERAGE_USER: 0.9983883, COVERAGE_USER_CORRECT: 0.9110623, DIVERSITY_GINI: 0.0112226, SHANNON_ENTROPY: 7.8944083, RATIO_DIVERSITY_HERFINDAHL: 0.9914788, RATIO_DIVERSITY_GINI: 0.0433335, RATIO_SHANNON_ENTROPY: 0.6344814, RATIO_AVERAGE_POPULARITY: 2.6745651, RATIO_NOVELTY: 0.0290592, 

EvaluatorHoldout: Processed 13640 (100.0%) in 15.35 sec. Users per second: 889
SearchBa

...

Iteration No: 39 ended. Search finished for the next optimal point.
Time taken: 29.5311
Function value obtained: -0.1187
Current minimum: -0.1187
Iteration No: 40 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.0, 'beta': 1.0, 'gamma': 1.0}
EvaluatorHoldout: Processed 13628 (100.0%) in 14.55 sec. Users per second: 937
SearchBayesianSkopt: Config 39 is suboptimal. Config: {'alpha': 0.0, 'beta': 1.0, 'gamma': 1.0} - results: PRECISION: 0.2438656, PRECISION_RECALL_MIN_DEN: 0.2477059, RECALL: 0.0690298, MAP: 0.1180898, MAP_MIN_DEN: 0.1196763, MRR: 0.4578615, NDCG: 0.2496831, F1: 0.1076014, HIT_RATE: 0.9175961, ARHR_ALL_HITS: 0.7339271, NOVELTY: 0.0055895, AVERAGE_POPULARITY: 0.5108856, DIVERSITY_MEAN_INTER_LIST: 0.9291400, DIVERSITY_HERFINDAHL: 0.9929072, COVERAGE_ITEM: 0.0776344, COVERAGE_ITEM_CORRECT: 0.0528268, COVERAGE_USER: 0.9983883, COVERAGE_USER_CORRECT: 0.9161172, DIVERSITY_GINI: 0.0130151, SHANNON_ENTROPY: 8.1535851, RATIO_DIVERSITY

Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 29.7280
Function value obtained: -0.1187
Current minimum: -0.1187
Iteration No: 46 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.0, 'beta': 0.2703351681126784, 'gamma': 1.0}
EvaluatorHoldout: Processed 13628 (100.0%) in 14.61 sec. Users per second: 933
SearchBayesianSkopt: New best config found. Config 45: {'alpha': 0.0, 'beta': 0.2703351681126784, 'gamma': 1.0} - results: PRECISION: 0.2450323, PRECISION_RECALL_MIN_DEN: 0.2487904, RECALL: 0.0690874, MAP: 0.1187130, MAP_MIN_DEN: 0.1202312, MRR: 0.4577386, NDCG: 0.2503777, F1: 0.1077847, HIT_RATE: 0.9178896, ARHR_ALL_HITS: 0.7352856, NOVELTY: 0.0056253, AVERAGE_POPULARITY: 0.5005800, DIVERSITY_MEAN_INTER_LIST: 0.9318927, DIVERSITY_HERFINDAHL: 0.9931824, COVERAGE_ITEM: 0.0991749, COVERAGE_ITEM_CORRECT: 0.0629603, COVERAGE_USER: 0.9983883, COVERAGE_USER_CORRECT: 0.9164103, DIVERSITY_GINI: 0.0147629, SHANNON_ENTR

Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 14.8846
Function value obtained: -0.1185
Current minimum: -0.1187
Iteration No: 52 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.6024086967798707, 'beta': 0.2538249863369079, 'gamma': 0.7660249894943786}
EvaluatorHoldout: Processed 13628 (100.0%) in 14.54 sec. Users per second: 937
SearchBayesianSkopt: Config 51 is suboptimal. Config: {'alpha': 0.6024086967798707, 'beta': 0.2538249863369079, 'gamma': 0.7660249894943786} - results: PRECISION: 0.2386997, PRECISION_RECALL_MIN_DEN: 0.2424617, RECALL: 0.0675407, MAP: 0.1136491, MAP_MIN_DEN: 0.1151590, MRR: 0.4432039, NDCG: 0.2425217, F1: 0.1052895, HIT_RATE: 0.9148077, ARHR_ALL_HITS: 0.7079243, NOVELTY: 0.0055722, AVERAGE_POPULARITY: 0.5413985, DIVERSITY_MEAN_INTER_LIST: 0.8930067, DIVERSITY_HERFINDAHL: 0.9892941, COVERAGE_ITEM: 0.0918656, COVERAGE_ITEM_CORRECT: 0.0588072, COVERAGE_USER: 0.9983883, COVERAGE_USER_

Iteration No: 58 ended. Search finished for the next optimal point.
Time taken: 14.9825
Function value obtained: -0.1138
Current minimum: -0.1187
Iteration No: 59 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.1273118559213315, 'beta': 0.5976700374977204, 'gamma': 1.0}
EvaluatorHoldout: Processed 13628 (100.0%) in 14.56 sec. Users per second: 936
SearchBayesianSkopt: Config 58 is suboptimal. Config: {'alpha': 0.1273118559213315, 'beta': 0.5976700374977204, 'gamma': 1.0} - results: PRECISION: 0.2440637, PRECISION_RECALL_MIN_DEN: 0.2477990, RECALL: 0.0689399, MAP: 0.1181563, MAP_MIN_DEN: 0.1197137, MRR: 0.4567121, NDCG: 0.2495700, F1: 0.1075114, HIT_RATE: 0.9175227, ARHR_ALL_HITS: 0.7330541, NOVELTY: 0.0055935, AVERAGE_POPULARITY: 0.5147047, DIVERSITY_MEAN_INTER_LIST: 0.9241395, DIVERSITY_HERFINDAHL: 0.9924072, COVERAGE_ITEM: 0.0852207, COVERAGE_ITEM_CORRECT: 0.0564262, COVERAGE_USER: 0.9983883, COVERAGE_USER_CORRECT: 0.9160440, DIVERSITY_

EvaluatorHoldout: Processed 13640 (100.0%) in 14.69 sec. Users per second: 929
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'alpha': 0.0, 'beta': 0.3023745211799622, 'gamma': 1.0} - results:
CUTOFF: 10 - PRECISION: 0.2912170, PRECISION_RECALL_MIN_DEN: 0.2942925, RECALL: 0.0705663, MAP: 0.1511817, MAP_MIN_DEN: 0.1524617, MRR: 0.5096775, NDCG: 0.2967496, F1: 0.1136045, HIT_RATE: 0.9474340, ARHR_ALL_HITS: 0.8730996, NOVELTY: 0.0056226, AVERAGE_POPULARITY: 0.5014181, DIVERSITY_MEAN_INTER_LIST: 0.9316227, DIVERSITY_HERFINDAHL: 0.9931554, COVERAGE_ITEM: 0.0990088, COVERAGE_ITEM_CORRECT: 0.0661720, COVERAGE_USER: 0.9992674, COVERAGE_USER_CORRECT: 0.9467399, DIVERSITY_GINI: 0.0146451, SHANNON_ENTROPY: 8.2804482, RATIO_DIVERSITY_HERFINDAHL: 0.9935291, RATIO_DIVERSITY_GINI: 0.0565491, RATIO_SHANNON_ENTROPY: 0.6655078, RATIO_AVERAGE_POPULARITY: 2.5154805, RATIO_NOVELTY: 0.0294166, 


SearchBayesianSkopt: Saving model in logs/MergeThreeModelsByScore

MergeThreeModelsByScore:

In [3]:
recommender = MergeThreeModelsByScores(ICM_combined,recommenders)
recommender.fit(alpha=0.0, beta=0.3023745211799622, gamma=1.0)

In [5]:
from Utils.utils import create_submission, write_submission

submission = create_submission(recommender)
write_submission(submission,"SLIME_RP3_IALS_Stacked_hybridBeta")